In [0]:
pip install hubspot

Python interpreter will be restarted.
Collecting hubspot
 Using cached hubspot-0.1.14.dev0-py3-none-any.whl
Installing collected packages: hubspot
Successfully installed hubspot-0.1.14.dev0
WARNING: You are using pip version 21.0.1; however, version 23.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3023b221-cbf0-4335-bc1c-434fae02e819/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
pip install hubspot-api-client

Python interpreter will be restarted.
Collecting hubspot-api-client
 Using cached hubspot_api_client-8.0.0-py3-none-any.whl (3.4 MB)
Requirement already satisfied: six>=1.10 in /databricks/python3/lib/python3.8/site-packages (from hubspot-api-client) (1.15.0)
Requirement already satisfied: python-dateutil in /databricks/python3/lib/python3.8/site-packages (from hubspot-api-client) (2.8.1)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.8/site-packages (from hubspot-api-client) (2020.12.5)
Requirement already satisfied: urllib3>=1.15 in /databricks/python3/lib/python3.8/site-packages (from hubspot-api-client) (1.25.11)
Installing collected packages: hubspot-api-client
Successfully installed hubspot-api-client-8.0.0
WARNING: You are using pip version 21.0.1; however, version 23.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3023b221-cbf0-4335-bc1c-434fae02e819/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import pandas as pd

In [0]:
export_companies = spark.read.csv('dbfs:/mnt/alphaplanexports/AdressenExport230712.csv', sep=';', header=True, encoding='latin1')

In [0]:
# companies_df = pd.read_csv('/dbfs/mnt/alphaplanexports/AdressenExport230706.CSV', sep=';',  encoding='latin1')

In [0]:
companies_df = export_companies.toPandas()

In [0]:
companies_df = companies_df.drop('SuchName', axis=1)

In [0]:
len(companies_df), len(companies_df[companies_df['Name1'].notnull()])

Out[12]: (8455, 8431)

In [0]:
def convert_scientific_notation(phone_value):
    phone_string = str(phone_value)
    if ('E+' in phone_string) or ('e+' in phone_string):
        phone_string = phone_string.replace(',','.')
        phone_float = float(phone_string)
        return str(phone_float).split('.')[0]

In [0]:
companies_df['Telefon'] = companies_df['Telefon'].apply(convert_scientific_notation)
companies_df['Telefax'] = companies_df['Telefax'].apply(convert_scientific_notation)

In [0]:
companies_df = companies_df[companies_df['Name1'].notnull()]

In [0]:
mapping = {'AdressenID': 'k_alphaplan_addressenid_hubspot',
           'AdressNummer': 'k_alphaplan_adressnummer_hubspot',
           'Name1' : 'name',
           'Ort' : 'city',
           'Postleitzahl' : 'zip',
           'Strasse' : 'address',
           'Telefon': 'phone',
           'LaenderKuerzel': 'laenderkuerzel',
            'Postfach': 'postfach',
            'PostfachPostleitzahl': 'postfach_postleitzahl',
            'PostfachOrt': 'postfach_ort',
            'Telefax': 'telefax' ,
            'Email': 'email',
            'Information': 'information',
            'Internet': 'website',
            'LieferantenKundenNummer': 'lieferanten_kunden_nummer',
            'PreisStufe1': 'preisstufe_1',
            'PreisStufe2': 'preisstufe_2',
            'SVPrioritaet': 'prioritaet'
}
companies_df = companies_df.rename(columns=mapping)

In [0]:
companies_records = companies_df.to_dict(orient='records')

In [0]:
n_records = len(companies_records)
print('--------companies records to be uploaded :',n_records)

--------companies records to be uploaded : 8431

In [0]:
def split_list_into_chunks(lst, chunk_size):
    """
    Split a list into chunks of a specified size.
    
    Args:
        lst (list): The list to split.
        chunk_size (int): The size of each chunk.
    
    Returns:
        list: A list of lists, where each inner list represents a chunk.
    """
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

In [0]:
companies_batches = split_list_into_chunks(companies_records, 100)

In [0]:
def get_hubspot_upload_format(batch):
    upload_list = []
    for batch_entry in batch:
        upload_list_entry = { "properties": batch_entry }
        upload_list.append(upload_list_entry)   
    return upload_list

In [0]:
import hubspot
from pprint import pprint
from hubspot.crm.companies import BatchInputSimplePublicObjectInputForCreate, ApiException

api_upload_key = dbutils.secrets.get(scope="key-vault-secrets",key="upload-api-accesstoken")

client = hubspot.Client.create(access_token=api_upload_key)

def upload_batch_companies(upload_list, client):
    batch_input_simple_public_object_input_for_create =   \
                    BatchInputSimplePublicObjectInputForCreate(inputs= upload_list
                                                                )
    try:
        api_response = client.crm.companies.batch_api.create(batch_input_simple_public_object_input_for_create=batch_input_simple_public_object_input_for_create)
        n_uploads = len(api_response.results)
        return n_uploads
    except ApiException as e:
        print("Exception when calling batch_api->create: %s\n" % e)

In [0]:
# test_batch = companies_batches[0][:30]
# upload_list = get_hubspot_upload_format(test_batch)
# n_uploads = upload_batch_companies(upload_list, client)
# print('--------companies records uploaded successfully:',n_uploads)

In [0]:
n_uploads = 0
counter = 1
for b in companies_batches:    
    upload_list = get_hubspot_upload_format(b)
    n_uploads_inc = upload_batch_companies(upload_list, client)
    print('-----uploaded batch :', counter, ',number of records :', n_uploads_inc)
    n_uploads += n_uploads_inc
    counter += 1

-----uploaded batch : 1 ,number of records : 100
-----uploaded batch : 2 ,number of records : 100
-----uploaded batch : 3 ,number of records : 100
-----uploaded batch : 4 ,number of records : 100
-----uploaded batch : 5 ,number of records : 100
-----uploaded batch : 6 ,number of records : 100
-----uploaded batch : 7 ,number of records : 100
-----uploaded batch : 8 ,number of records : 100
-----uploaded batch : 9 ,number of records : 100
-----uploaded batch : 10 ,number of records : 100
-----uploaded batch : 11 ,number of records : 100
-----uploaded batch : 12 ,number of records : 100
-----uploaded batch : 13 ,number of records : 100
-----uploaded batch : 14 ,number of records : 100
-----uploaded batch : 15 ,number of records : 100
-----uploaded batch : 16 ,number of records : 100
-----uploaded batch : 17 ,number of records : 100
-----uploaded batch : 18 ,number of records : 100
-----uploaded batch : 19 ,number of records : 100
-----uploaded batch : 20 ,number of records : 100
-----uploaded batch : 21 ,number of records : 100
-----uploaded batch : 22 ,number of records : 100
-----uploaded batch : 23 ,number of records : 100
-----uploaded batch : 24 ,number of records : 100
-----uploaded batch : 25 ,number of records : 100
-----uploaded batch : 26 ,number of records : 100
-----uploaded batch : 27 ,number of records : 100
-----uploaded batch : 28 ,number of records : 100
-----uploaded batch : 29 ,number of records : 100
-----uploaded batch : 30 ,number of records : 100
-----uploaded batch : 31 ,number of records : 100
-----uploaded batch : 32 ,number of records : 100
-----uploaded batch : 33 ,number of records : 100
-----uploaded batch : 34 ,number of records : 100
-----uploaded batch : 35 ,number of records : 100
-----uploaded batch : 36 ,number of records : 100
-----uploaded batch : 37 ,number of records : 100
-----uploaded batch : 38 ,number of records : 100
-----uploaded batch : 39 ,number of records : 100
-----uploaded batch : 40 ,number of records : 100
-----uploaded batch : 41 ,number of records : 100
-----uploaded batch : 42 ,number of records : 100
-----uploaded batch : 43 ,number of records : 100
-----uploaded batch : 44 ,number of records : 100
-----uploaded batch : 45 ,number of records : 100
-----uploaded batch : 46 ,number of records : 100
-----uploaded batch : 47 ,number of records : 100
-----uploaded batch : 48 ,number of records : 100
-----uploaded batch : 49 ,number of records : 100
-----uploaded batch : 50 ,number of records : 100
-----uploaded batch : 51 ,number of records : 100
-----uploaded batch : 52 ,number of records : 100
-----uploaded batch : 53 ,number of records : 100
-----uploaded batch : 54 ,number of records : 100
-----uploaded batch : 55 ,number of records : 100
-----uploaded batch : 56 ,number of records : 100
-----uploaded batch : 57 ,number of records : 100
-----uploaded batch : 58 ,number of records : 100
-----uploaded batch : 59 ,number of records : 100
-----uploaded batch : 60 ,number of records : 100
-----uploaded batch : 61 ,number of records : 100
-----uploaded batch : 62 ,number of records : 100
-----uploaded batch : 63 ,number of records : 100
-----uploaded batch : 64 ,number of records : 100
-----uploaded batch : 65 ,number of records : 100
-----uploaded batch : 66 ,number of records : 100
-----uploaded batch : 67 ,number of records : 100
-----uploaded batch : 68 ,number of records : 100
-----uploaded batch : 69 ,number of records : 100
-----uploaded batch : 70 ,number of records : 100
-----uploaded batch : 71 ,number of records : 100
-----uploaded batch : 72 ,number of records : 100
-----uploaded batch : 73 ,number of records : 100
-----uploaded batch : 74 ,number of records : 100
-----uploaded batch : 75 ,number of records : 100
-----uploaded batch : 76 ,number of records : 100
-----uploaded batch : 77 ,number of records : 100
-----uploaded batch : 78 ,number of records : 100
-----uploaded batch : 79 ,number of records : 100
-----uploaded batch : 80 ,number of records : 100
-----uplo

In [0]:
print('--------companies records uploaded successfully:',n_uploads)

--------companies records uploaded successfully: 8431